### Dec 22nd 2015 Riddler: How Long Will Your Smartphone Distract You From Family Dinner? 
***

The Dec 22nd 2015 Riddler from [FiveThirtyEight](https://fivethirtyeight.com/features/how-long-will-your-smartphone-distract-you-from-family-dinner/) asks the following: 

> You’ve just finished unwrapping your holiday presents. You and your sister got brand-new smartphones, opening them at the same moment. You immediately both start doing important tasks on the Internet, and each task you do takes one to five minutes. (All tasks take exactly one, two, three, four or five minutes, with an equal probability of each). After each task, you have a brief moment of clarity. During these, you remember that you and your sister are supposed to join the rest of the family for dinner and that you promised each other you’d arrive together. You ask if your sister is ready to eat, but if she is still in the middle of a task, she asks for time to finish it. In that case, you now have time to kill, so you start a new task (again, it will take one, two, three, four or five minutes, exactly, with an equal probability of each). If she asks you if it’s time for dinner while you’re still busy, you ask for time to finish up and she starts a new task and so on. From the moment you first open your gifts, how long on average does it take for both of you to be between tasks at the same time so you can finally eat? (You can assume the “moments of clarity” are so brief as to take no measurable time at all.) 

**Simulation Approach**: Here we run many trials of the smartphone game and use Monte Carlo to estimate the expected time-to-dinner. 

In [3]:
using Distributions

In [ ]:
function smartphone_trial(verbose=false)
    # ------------------------------------------------------------
    # Function to run one trial of the smartphone game 
    # ------------------------------------------------------------
    
    # Define U[1,5] distribution 
    u15 = DiscreteUniform(1,5)
    
    # Initialize tasks 
    state = rand(u15, 2)
    
    if verbose
        @printf "r%1d |  0  0  | \n" state[1] 
        @printf "   | %2d  0  | r%1d\n" state[1] state[2] 
    end
    
    # Until synced, advance idle person by sample from U[1,5]
    while state[1] != state[2]
        roll = rand(u15)
        if verbose
            if state[1] < state[2]
                @printf "r%1d" roll
            else
                @printf "  " 
            end
            @printf " | %2d %2d  |" state[1] state[2]
            if state[2] < state[1]
                @printf " r%1d\n" roll
            else
                @printf "  \n" 
            end
        end
        state += (state[1] < state[2]) ? [roll, 0] : [0, roll]
    end
    
    if verbose
        @printf "   | %2d %2d  |" state[1] state[2]
    end
    
    return state[1]
end

function smartphone_sim(M::Int64=1000000)
    # ------------------------------------------------------------
    # Function to run many trials of the game and 
    # approximate the expected end-time
    # ------------------------------------------------------------
    
    cumu_time = 0
    
    for ii=1:M 
        cumu_time += smartphone_trial()
    end 
    
    exp_time = cumu_time / Float64(M)
    
    return exp_time 
    
end

Here's one trial of the smartphone game showing the running cumulative task time along with the value of the random task selection. 

In [8]:
smartphone_trial(true);

r5 |  0  0  | 
   |  5  0  | r2
   |  5  2  | r5
r4 |  5  7  |  
   |  9  7  | r5
r2 |  9 12  |  
r5 | 11 12  |  
   | 16 12  | r5
r5 | 16 17  |  
   | 21 17  | r3
   | 21 20  | r5
r3 | 21 25  |  
r2 | 24 25  |  
   | 26 25  | r3
r5 | 26 28  |  
   | 31 28  | r3
   | 31 31  |

We simulate 10 Million trials of the game and approximated the expected time (in minutes) until the game ends. 

In [5]:
@printf "E[time to end] \U02248 %.5f" smartphone_sim(Int64(1e7))

E[time to end] ≈ 9.00063

Looks like the expected sync time is 9 minutes.  Let's see if we can back that up. 

**Analytic Approach**: We will represent the game and the subsequent sequence of random tasks as a stochastic process $\{X_n: n\geq 1\}$ where each $X_n$ is an i.i.d. draw from $\textrm{unif}(1,5)$.  To make the game fit this framework we will assume that at the beginning of the game one person selects a random task, and then, being idle, the other person selects a random task.    

Notice that when the two players' task times sync up and the game ends, the total time expended will be equal to one half the value of the sum of each of the random tasks.  This assumption makes things easier because we don't have to worry about assigning each random task selection to a particular player.  

Let $\tau$ be the random time at which the game stops (where here *time* is really the number of randomly selected tasks).  Then Wald's equation tells us that 

$$
\frac{1}{2}E\left[ \sum_{n=1}^\tau X_n\right] = \frac{1}{2}~E[\tau]~E[X]
$$

Computing $E[X]$ is easy.  It's simply the expected value of the random variable $X$ which is $\texttt{unif}(1,5)$. 

$$
E[X] = \frac{1 + 5}{2} = 3
$$

Computing $E[\tau]$ requires a bit more work. We will begin by computing the probability that the game ends after a particular number of random tasks have been selected.  

- The probability that the game ends after the first random task is zero. 

<p> </p> 

- The probability that the game ends after the second random task is selected is $\frac{1}{5}$.  To see this note that the first random task, $X_1$, is an integer between $1$ and $5$.  The game ends on the selection of $X_2$ if the value of $X_2$ is equal to the value of $X_1$, which happens with probability $p=\frac{1}{5}$. 

<p> </p> 

- The game ends on the third random task if $X_3$ is chosen to be the the time remaining on the task of the currently active player.  Since the time remaining (if the game is still going) is always an integer between $1$ and $4$  the probability of $X_3$ begin chosen to be exactly the remaining time is again $p=\frac{1}{5}$.  So, the probability that the game ends on the third random task is equal to the probability that the game did **not** end on the second random task times the probability that $X_3$ is chosen to be the remaining time.  This probability is $(1-p)p$. 

<p> </p> 

- Similarly, the probability that the game ends on the fourth random tasks is equal to the probability that it did not end on tasks 2 and 3, and ends on task 4.  This probability is $(1-p)^2p$. 

You get the idea.  So, the expected stopping time is equal to the infinite sum of the probability of the game ending on a particular task times the number of tasks that have occurred up to that point.  Thus, we have

$$
E\left[\tau\right] = 2p + 3(1-p)p + 4(1-p)^2p + 5(1-p)^3p + \cdots = \sum_{k=1}^\infty (k+1)(1-p)^{k-1}p
= 1 + \frac{1}{p} = 6
$$

Plugging the expected stopping time and the expected value of $X$ into our expression from Wald's Equation gives 

$$
E[\texttt{total time}] = 
\frac{1}{2}E\left[ \sum_{n=1}^\tau X_n\right] = \frac{1}{2}~E[\tau]~E[X] = \frac{1}{2} \cdot 6 \cdot 3 = 9 
$$

which agrees with our numerical simulation. 

### Appendix
***

**The Expected Stopping Time Calculation**: 

\begin{eqnarray}
\nonumber E\left[\tau\right] &=& 2p + 3(1-p)p + 4(1-p)^2p + 5(1-p)^3p + \cdots \\
\nonumber &=& \sum_{k=1}^\infty (k+1)(1-p)^{k-1}p \\
\nonumber &=& \sum_{k=1}^\infty (1-p)^{k-1}p + \sum_{k=1}^\infty k(1-p)^{k-1}p \\
\nonumber &=& p\sum_{n=0}^\infty (1-p)^{n} + \sum_{k=1}^\infty k(1-p)^{k-1}p \\
\nonumber &=& p\frac{1}{1-(1-p)} + \sum_{k=1}^\infty k(1-p)^{k-1}p \\
\nonumber &=& 1 + \sum_{k=1}^\infty k(1-p)^{k-1}p \\
\nonumber &=& 1 + \sum_{m=0}^\infty (m+1)(1-p)^{m}p \\
\nonumber &=& 1 + p + (1-p)\sum_{m=1}^\infty (m+1)(1-p)^{m-1}p \\ 
\nonumber &=& 1 + p + (1-p)E[\tau]
\end{eqnarray}


Thus we have 

$$
E\left[\tau \right] = 1 + p + (1-p)E\left[ \tau\right] \quad \Rightarrow \quad pE\left[\tau\right] = 1+p \quad \Rightarrow \quad E[\tau] = 1 + \frac{1}{p} 
$$